In [1]:
import pandas as pd
from datetime import datetime

In [ ]:
#flights = pd.read_parquet('data/flight_data.parquet')
#weather = pd.read_parquet('weather.parquet')



In [4]:
# First, let's drop the rows with null values in Origin or DepTime from flights
flights = flights.dropna(subset=['Origin', 'CRSDepTime'])

# Convert both category columns to the same dtype
flights["Origin"] = flights["Origin"].astype(str)
weather['IATA'] = weather['IATA'].astype(str)

# Create the datetime columns
mins = pd.to_timedelta(flights["CRSDepTime"] // 100 * 60 + flights["CRSDepTime"] % 100, unit='m')
flights["dep_datetime"] = pd.to_datetime(flights["FlightDate"]) + mins
weather["date"] = pd.to_datetime(weather["date"].apply(lambda x: x.replace(tzinfo=None)))

# Sort the DataFrames
flights.sort_values(by=["dep_datetime"], inplace=True)
weather.sort_values(by=["date"], inplace=True)

# Perform the merge
merged = pd.merge_asof(flights, weather,
                      left_by='Origin', left_on='dep_datetime',
                      right_by='IATA', right_on='date', direction='nearest')

# Drop the redundant columns
merged.drop(columns=['FlightDate', 'IATA', 'date'], inplace=True)

In [ ]:
# save merged to parquet

#merged.to_parquet('merged_data.parquet')

In [ ]:
def merge_flight_weather_on_dest_using_deptime(merged, weather):
    # Make copies to avoid modifying the original dataframes

    # Convert both category columns to the same dtype
    merged["Dest"] = merged["Dest"].astype(str)
    weather['IATA'] = weather['IATA'].astype(str)
    
    # Create the datetime columns based on departure time
    # We already have the departure datetime, so we'll use that
    merged["dest_dep_datetime"] = merged["dep_datetime"]
    
    # Prepare weather data (should already be datetime from previous code)
    weather["date"] = pd.to_datetime(weather["date"].apply(lambda x: x.replace(tzinfo=None)))
    
    # Sort the DataFrames (required for merge_asof)
    merged.sort_values(by=["dest_dep_datetime"], inplace=True)
    weather.sort_values(by=["date"], inplace=True)
    
    # Rename weather columns to add _dest_dep_time suffix (except for merge columns)
    weather_renamed = weather.copy()
    for col in weather.columns:
        if col not in ['IATA', 'date']:
            weather_renamed.rename(columns={col: f"{col}_dest_dep_time"}, inplace=True)
    
    # Perform the merge
    result = pd.merge_asof(merged, weather_renamed,
                          left_by='Dest', left_on='dest_dep_datetime',
                          right_by='IATA', right_on='date', direction='nearest')
    
    # Drop redundant columns
    if 'IATA' in result.columns:
        result.drop(columns=['IATA'], inplace=True)
    if 'date' in result.columns:
        result.drop(columns=['date'], inplace=True)
    
    return result

def merge_flight_weather_on_dest_using_deptime(merged, weather):
    # Make copies to avoid modifying the original dataframes

    # Convert both category columns to the same dtype
    merged["Dest"] = merged["Dest"].astype(str)
    weather['IATA'] = weather['IATA'].astype(str)
    
    # Create the datetime columns based on departure time
    # We already have the departure datetime, so we'll use that
    merged["dest_dep_datetime"] = merged["dep_datetime"]
    
    # Prepare weather data (should already be datetime from previous code)
    weather["date"] = pd.to_datetime(weather["date"].apply(lambda x: x.replace(tzinfo=None)))
    
    # Sort the DataFrames (required for merge_asof)
    merged.sort_values(by=["dest_dep_datetime"], inplace=True)
    weather.sort_values(by=["date"], inplace=True)
    
    # Rename weather columns to add _dest_dep_time suffix (except for merge columns)
    weather_renamed = weather.copy()
    for col in weather.columns:
        if col not in ['IATA', 'date']:
            weather_renamed.rename(columns={col: f"{col}_dest_dep_time"}, inplace=True)
    
    # Perform the merge
    result = pd.merge_asof(merged, weather_renamed,
                          left_by='Dest', left_on='dest_dep_datetime',
                          right_by='IATA', right_on='date', direction='nearest')
    
    # Drop redundant columns
    if 'IATA' in result.columns:
        result.drop(columns=['IATA'], inplace=True)
    if 'date' in result.columns:
        result.drop(columns=['date'], inplace=True)
    
    return result


In [ ]:
#merged = pd.read_parquet('merged_data.parquet')

In [ ]:
#merged = merge_flight_weather_on_dest_using_deptime(merged, weather)

In [3]:
merged = pd.read_parquet('data/WEATHER.parquet')

In [12]:
merged

,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,...,wind_direction_10m_dest_dep_time,wind_direction_80m_dest_dep_time,wind_direction_120m_dest_dep_time,wind_direction_180m_dest_dep_time,wind_gusts_10m_dest_dep_time,temperature_80m_dest_dep_time,temperature_120m_dest_dep_time,temperature_180m_dest_dep_time,latitude_dest_dep_time,longitude_dest_dep_time
0,2023,1,1,1,7,AS,AS,19930,AS,150.0,...,194.676440,195.25517,181.82405,181.56220,45.000000,13.633500,13.483500,12.6235,33.942501,-118.407997
1,2023,1,1,1,7,F9,F9,20436,F9,1524.0,...,216.869980,252.52513,258.05585,260.70676,15.840000,13.115500,13.315499,12.3220,33.636700,-84.428101
2,2023,1,1,1,7,DL,DL,19790,DL,1099.0,...,202.249070,203.86018,205.97433,228.36655,29.519999,-0.983000,-1.083000,-0.3480,44.882000,-93.221802
3,2023,1,1,1,7,B6,B6,20409,B6,956.0,...,216.027470,224.99990,247.79646,265.05410,16.199999,9.760500,9.860500,10.9780,40.777199,-73.872597
4,2023,1,1,1,7,AA,AA,19805,AA,500.0,...,111.801476,135.85501,159.14546,164.84589,17.640000,23.776000,23.675999,22.1065,25.793200,-80.290604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14648895,2024,4,12,31,2,UA,UA,19977,UA,2379.0,...,120.963690,125.13428,145.61960,104.53450,11.879999,14.712999,14.513000,13.0065,29.984400,-95.341400
14648896,2024,4,12,31,2,AA,AA,19805,AA,508.0,...,202.833700,207.82400,244.98314,304.38040,11.520000,5.693500,5.893500,5.5935,35.214001,-80.943100
14648897,2024,4,12,31,2,DL,DL,19790,DL,929.0,...,302.471160,305.21768,323.39282,323.13000,11.520000,6.165500,5.965500,4.6220,33.636700,-84.428101
14648898,2024,4,12,31,2,AA,AA,19805,AA,2793.0,...,202.833700,207.82400,244.98314,304.38040,11.520000,5.693500,5.893500,5.5935,35.214001,-80.943100


In [15]:
merged = merge_flight_weather_on_dest_using_arrtime(merged, weather)

: 

In [7]:
# Save the merged DataFrame

merged.to_parquet('data/WEATHER.parquet')